# Final Project

Group Members
- Chuyang Xiao
- Cody Smith
- Jason Lin
- Matthew Montehermoso

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import io, signal
import pandas as pd

In [2]:
monkey_ecog_data = io.loadmat('ECoG_monkey/ECoG_monkey.mat', squeeze_me = True)
print(monkey_ecog_data.keys())
print(monkey_ecog_data['fs'])
print(monkey_ecog_data['ecog_eyesopen'].shape)
print(monkey_ecog_data['labels'])
print(monkey_ecog_data['elec_num'])

plt.figure(figsize=(15,3))
plt.plot(np.arange(0,monkey_ecog_data['ecog_eyesopen'].shape[1]/monkey_ecog_data['fs'],1/monkey_ecog_data['fs']), 
         monkey_ecog_data['ecog_eyesopen'].T)
plt.xlim([0,10])
plt.xlabel('Time (s)')
plt.legend(monkey_ecog_data['labels'])

FileNotFoundError: [Errno 2] No such file or directory: 'ECoG_monkey/ECoG_monkey.mat'

In [ ]:
# the following code will be used to assemble the appropriate code into a DataFrame
# so that it is a little easier to look at


# given labels and data
def to_DataFrame(state_data, label_data):
    # initialize temporary dictionary to poull from
    temp_dict = dict()
    
    # loop through values of length 3 (the 3 labels, occipital, cingulate, temporal)
    for i in range(len(label_data)):
        # store i-th label 
        label = label_data[i]
        # store i-th list (which has been converted into an np.array()
        temp_data = np.array(state_data[i,:])
        # store new label and item as a dictionary pair
        temp_dict[label] = temp_data

    # convert into DataFrame object
    as_DataFrame = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in temp_dict.items() ]))
    
    return as_DataFrame

ecog_open = to_DataFrame(monkey_ecog_data['ecog_eyesopen'], monkey_ecog_data['labels'].T )
ecog_close = to_DataFrame(monkey_ecog_data['ecog_eyesclosed'], monkey_ecog_data['labels'] )
ecog_anes = to_DataFrame(monkey_ecog_data['ecog_anes'], monkey_ecog_data['labels'] )


In [ ]:
print(ecog_open.head())
print(ecog_close.head())
print(ecog_anes.head())
print(np.fft.fft(ecog_open['cingulate']))

In [ ]:
# intializing fs for our data
monkey_fs = monkey_ecog_data['fs']

# time vector in seconds
t_monkey = np.arange(0, 300, 1/monkey_fs )

freq_monkey = np.fft.fftfreq(300000, 1/monkey_fs)
print(freq_monkey.shape)

# compute FT of the DataFrame
# note that the inputs HAVE to be DataFrames
def compute_FT(state_data):
    # obtain column names of dataframes
    labels = list(state_data.columns.values)
    temp_dict = dict()
    
    for label in labels:
        temp_FT = np.fft.fft(state_data[label])
        # store label and item as a dictionary pair
        temp_dict[label] = temp_FT
        
    as_DataFrame = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in temp_dict.items() ]))
    
    return as_DataFrame
    
    
open_FT = compute_FT(ecog_open)
close_FT = compute_FT(ecog_close)
anes_FT = compute_FT(ecog_anes)


plt.figure(figsize = (24,36))
plt.subplot(3,1,1)
plt.plot(freq_monkey, np.log10(np.abs(open_FT['occipital'])**2), '.', label = 'Eyes Open')
plt.plot(freq_monkey, np.log10(np.abs(close_FT['occipital'])**2), '.', label = 'Eye Closed')
plt.plot(freq_monkey, np.log10(np.abs(anes_FT['occipital'])**2), '.', label = 'Anesthesia')
plt.xlim([0,15])
plt.title("Power Spectrum - Occipital Lobe")
plt.ylabel('Power'); plt.xlabel("Freqeuncy (in Hz)")
plt.legend()
plt.figure(figsize = (24,36))
plt.subplot(3,1,2)
temp_open_power = np.abs(open_FT['temporal']) **2
temp_close_power = np.abs(close_FT['temporal']) **2
temp_ane_power = np.abs(anes_FT['temporal']) **2
plt.plot(freq_monkey, np.log10(temp_open_power), '.', label = 'Eye Open')
plt.plot(freq_monkey, np.log10(temp_close_power), '.',label = 'Eye Closed')
plt.plot(freq_monkey, np.log10(temp_ane_power), '.', label = 'Anethesia')
plt.xlim([0,15])
plt.title("Power Spectrum - Temporal Lobe")
plt.ylabel('Power'); plt.xlabel("Freqeuncy (in Hz)")
plt.legend()



# Filtering Our Data

When observing the Power Spectrum that corresponds to the Occipital Lobe, we find that the occipital lobe generally has high activity at lower frequencies, from around 0 Hz to around 10 Hz. The Power Spectrum that corresponds to the Temporal Lobe shows that there is high activity from the frequencies 0 Hz to around 15 Hz. Lastly,  In the Cingulate Power Spectrum it generally shows that there is high activity from 0 Hz to around 8 Hz. We can thus use, these found frequencies to produce our bandpass filters.

In [ ]:
# although the occipital and cingulate frequencies are higher by a little bit, the bulk of frequencies 
# found in the power spectrum data corresponds strongly with those of theta waves
occ_filt = signal.firwin( 30000, cutoff = 10, fs = monkey_fs, pass_zero = True)
temp_filt = signal.firwin( 30000, cutoff = 15, fs = monkey_fs, pass_zero = True)
cing_filt = signal.firwin( 30000, cutoff = 5, fs = monkey_fs, pass_zero = True)

#convolve corresponding state data with appropriate filters

# occiptal convolution w/ occipital filter
occ_open_osc = np.convolve(ecog_open['occipital'], occ_filt, mode = 'same')
occ_close_osc = np.convolve(ecog_close['occipital'], occ_filt, mode = 'same')
occ_anes_osc = np.convolve(ecog_anes['occipital'], occ_filt, mode = 'same')

# temporal convolution w/ temporal filter
temp_open_osc = np.convolve(ecog_open['temporal'], temp_filt, mode = 'same')
temp_close_osc = np.convolve(ecog_close['temporal'], temp_filt, mode = 'same')
temp_anes_osc = np.convolve(ecog_anes['temporal'], temp_filt, mode = 'same')

# temporal convolution w/ cingulate filter
cing_open_osc = np.convolve(ecog_open['cingulate'], cing_filt, mode = 'same')
cing_close_osc = np.convolve(ecog_close['cingulate'], cing_filt, mode = 'same')
cing_anes_osc = np.convolve(ecog_anes['cingulate'], cing_filt, mode = 'same')

In [ ]:
# plot the occipital signals and their filters

plt.figure(figsize = (20,20))

# occipital with opened eyes 
plt.subplot(3,1,1)
plt.plot(t_monkey, ecog_open['occipital'] , label = 'Original', color = 'red')
plt.plot(t_monkey, occ_open_osc, label = 'Filtered', color = 'black')
plt.xlim([0,15])
plt.title("Filtered Occipital Signal (Open Eyes)")
plt.ylabel('Voltage'); plt.xlabel("Time (in Seconds)")
plt.legend()

# occipital with closed eyes
plt.subplot(3,1,2)
plt.plot(t_monkey, ecog_close['occipital'] , label = 'Original', color = 'red')
plt.plot(t_monkey, occ_close_osc, label = 'Filtered', color = 'black')
plt.xlim([0,15])
plt.title("Filtered Occipital Signal (Closed Eyes)")
plt.ylabel('Voltage'); plt.xlabel("Time (in Seconds)")
plt.legend()
plt.subplot(3,1,3)

# occipital with anesthesia
plt.plot(t_monkey, ecog_anes['occipital'] , label = 'Original', color = 'red')
plt.plot(t_monkey, occ_anes_osc, label = 'Filtered', color = 'black')
plt.xlim([0,15])
plt.title("Filtered Occipital Signal (Anesthesia)")
plt.ylabel('Voltage'); plt.xlabel("Time (in Seconds)")
plt.legend()

In [ ]:
# plot the temporal signals and their filters

plt.figure(figsize = (20,20))

# temporal with opened eyes 
plt.subplot(3,1,1)
plt.plot(t_monkey, ecog_open['temporal'] , label = 'Original', color = 'orange')
plt.plot(t_monkey, temp_open_osc, label = 'Filtered', color = 'black')
plt.xlim([0,15])
plt.title("Filtered Temporal Signal (Open Eyes)")
plt.ylabel('Voltage'); plt.xlabel("Time (in Seconds)")
plt.legend()

# temporal with closed eyes
plt.subplot(3,1,2)
plt.plot(t_monkey, ecog_close['temporal'] , label = 'Original', color = 'orange')
plt.plot(t_monkey, temp_close_osc, label = 'Filtered', color = 'black')
plt.xlim([0,15])
plt.title("Filtered Temporal Signal (Closed Eyes)")
plt.ylabel('Voltage'); plt.xlabel("Time (in Seconds)")
plt.legend()
plt.subplot(3,1,3)

# temporal with anesthesia
plt.plot(t_monkey, ecog_anes['temporal'] , label = 'Original', color = 'orange')
plt.plot(t_monkey, temp_anes_osc, label = 'Filtered', color = 'black')
plt.xlim([0,15])
plt.title("Filtered Temporal Signal (Anesthesia)")
plt.ylabel('Voltage'); plt.xlabel("Time (in Seconds)")
plt.legend()

In [ ]:
# plot the cingulate signals and their filters

plt.figure(figsize = (20,20))

# temporal with opened eyes 
plt.subplot(3,1,1)
plt.plot(t_monkey, ecog_open['cingulate'] , label = 'Original', color = 'violet')
plt.plot(t_monkey, cing_open_osc, label = 'Filtered', color = 'black')
plt.xlim([0,15])
plt.title("Filtered Cingulate Signal (Open Eyes)")
plt.ylabel('Voltage'); plt.xlabel("Time (in Seconds)")
plt.legend()

# temporal with closed eyes
plt.subplot(3,1,2)
plt.plot(t_monkey, ecog_close['cingulate'] , label = 'Original', color = 'violet')
plt.plot(t_monkey, cing_close_osc, label = 'Filtered', color = 'black')
plt.xlim([0,15])
plt.title("Filtered Temporal Signal (Closed Eyes)")
plt.ylabel('Voltage'); plt.xlabel("Time (in Seconds)")
plt.legend()
plt.subplot(3,1,3)

# temporal with anesthesia
plt.plot(t_monkey, ecog_anes['cingulate'] , label = 'Original', color = 'violet')
plt.plot(t_monkey, cing_anes_osc, label = 'Filtered', color = 'black')
plt.xlim([0,15])
plt.title("Filtered Temporal Signal (Anesthesia)")
plt.ylabel('Voltage'); plt.xlabel("Time (in Seconds)")
plt.legend()

In [ ]:
def plot_spectrogram(spg, t, f, freq_lims=[0,100], plot_db=False):
    """
    Utility function for plotting the spectrogram for you.
    
    spg: spectrogram, 2D real-numbered array, dimensions are [frequency x time]
    t: time axis of spectrogram
    f: frequency axis of spectrogram
    freq_lims (optional): limits the frequency axis, defaults to 0-100Hz
    """
    plt.figure(figsize=(15,4))
    if plot_db:
        plt.imshow(10*np.log10(spg), aspect='auto', extent=[t[0], t[-1], f[-1], f[0]])
    else:
        plt.imshow(spg, aspect='auto', extent=[t[0], t[-1], f[-1], f[0]])
    plt.xlabel('Time'); plt.ylabel('Frequency(Hz)');
    plt.ylim(freq_lims)
    plt.colorbar()
    plt.tight_layout()

def slide_window_time(T,len_win,len_overlap):
    # T is total signal time, len_win is window length in seconds, len_overlap is overlap length in seconds
    time = 0
    t_steps = []
    #while the timestamp is at least len_win less than the total time append the time
    while (time + len_win) <= T:
        t_steps.append(time)
        time = time + (len_win - len_overlap)
    t_steps = np.array(t_steps)
    return t_steps
    
def slide_window_index(T,fs,len_win,len_overlap):
    ti = slide_window_time(T, len_win, len_overlap)
    index = ti*fs
    index = np.array(index).astype(int)
    return index
  
    
def my_stft(data, fs, len_win, len_overlap):
    T = data.size/fs
    inds_stft = slide_window_index(T,fs,len_win,len_overlap)
    t_stft = slide_window_time(T, len_win, len_overlap)
    f_stft = np.fft.fftfreq(int(len_win*fs), 1/fs)
    stft = np.array([np.fft.fft(data[i:i+int(len_win*fs)]) for i in inds_stft])
    stft = stft.T

    # clip the frequency axis to return just the non-negative frequencies
    # np.fft.fftfreq returns the nyquist frequency as negative, which we also need to keep
    positive_fs = np.logical_or(f_stft>=0, f_stft==-fs/2)    
    
    # I return for you just the positive frequencies 
    return abs(f_stft[positive_fs]), t_stft, stft[positive_fs, :]

In [ ]:
len_win = 5
len_overlap = 1


f_stft, t_stft, stft = my_stft(ecog_open['temporal'], monkey_fs, len_win, len_overlap)
spg = np.abs(stft)**2
psd_open_temp = spg.mean(axis = 1)
plot_spectrogram(spg, t_stft, f_stft, freq_lims=[0,10], plot_db=False)
plt.title('My Spectrogram Temporal Open')


len_win = 5
len_overlap = 1

f_stft, t_stft, stft = my_stft(ecog_close['temporal'], monkey_fs, len_win, len_overlap)
spg = np.abs(stft)**2
psd_close_temp = spg.mean(axis = 1)
plot_spectrogram(spg, t_stft, f_stft, freq_lims=[0,10], plot_db=False)
plt.title('My Spectrogram Temporal Closed')



len_win = 5
len_overlap = 1

f_stft, t_stft, stft = my_stft(ecog_anes['temporal'], monkey_fs, len_win, len_overlap)
spg = np.abs(stft)**2
psd_anes_temp = spg.mean(axis = 1)
plot_spectrogram(spg, t_stft, f_stft, freq_lims=[0,10], plot_db=False)
plt.title('My Spectrogram Temporal Anes')





In [ ]:
len_win = 5
len_overlap = 1


f_stft, t_stft, stft = my_stft(ecog_open['occipital'], monkey_fs, len_win, len_overlap)
spg = np.abs(stft)**2
psd_open_occi = spg.mean(axis = 1)
plot_spectrogram(spg, t_stft, f_stft, freq_lims=[0,10], plot_db=False)
plt.title('My Spectrogram Occipital Open')


len_win = 5
len_overlap = 1

f_stft, t_stft, stft = my_stft(ecog_close['temporal'], monkey_fs, len_win, len_overlap)
spg = np.abs(stft)**2
psd_close_occi = spg.mean(axis = 1)
plot_spectrogram(spg, t_stft, f_stft, freq_lims=[0,10], plot_db=False)
plt.title('My Spectrogram Occipital Closed')



len_win = 5
len_overlap = 1

f_stft, t_stft, stft = my_stft(ecog_anes['temporal'], monkey_fs, len_win, len_overlap)
spg = np.abs(stft)**2
psd_anes_occi = spg.mean(axis = 1)
plot_spectrogram(spg, t_stft, f_stft, freq_lims=[0,10], plot_db=False)
plt.title('My Spectrogram Occipital Anes')





In [ ]:
# plot the PSDs
plt.figure(figsize=(5,5))
plt.plot(f_stft, np.log10(psd_open_temp), label = 'PSD Temporal Open')
plt.plot(f_stft, np.log10(psd_close_temp), label = 'PSD Temporal Closed')
plt.plot(f_stft, np.log10(psd_anes_temp), label = 'PSD Temporal Anes')
plt.xlim([0,20])
plt.xlabel('Freq (Hz)')
plt.ylabel('Power')
plt.legend()

# plot the PSDs
plt.figure(figsize=(5,5))
plt.plot(f_stft, np.log10(psd_open_occi), label = 'PSD Occipital Open')
plt.plot(f_stft, np.log10(psd_close_occi), label = 'PSD Occipital Closed')
plt.plot(f_stft, np.log10(psd_anes_occi), label = 'PSD Occipital Anes')
plt.xlim([0,50])
plt.xlabel('Freq (Hz)')
plt.ylabel('Power')
plt.legend()